In [1]:
%matplotlib qt
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import os.path as op

from scipy.signal import hilbert
from PyEMD import EMD

# plt.rcParams['font.size'] = 14

In [2]:
def perform_EMD(x, plot_emd=None):
    '''
    Perform empirical mode decomposition on signal block 'x'
    '''
    # EMD
    emd_decomp = EMD()
    imfs = emd_decomp(x)
    
    # Visualize EMD
    if plot_emd and plot_emd is not None:
        plt.figure(figsize=(12, 12))
        for i in range(len(imfs)-1):
            plt.subplot(len(imfs)+1, 1, i+1)
            plt.plot(t, x, color='0.8')
            plt.plot(t, imfs[i], 'k')
            plt.xlim([np.min(t), np.max(t)])
            plt.ylabel('IMF ' + str(i + 1))
        plt.subplot(len(imfs)+1, 1, i+2)
        plt.plot(t, x, color='0.8')
        plt.plot(t, imfs[-1], 'k')
        plt.xlim([np.min(t), np.max(t)])
        plt.ylabel('Residual')
        plt.xlabel('Time (s)')
        plt.tight_layout()
        plt.show()
    return imfs

def downsample_rows(arr, k):
    res = np.cumsum(arr, 0)[k-1::k]
    res[1:] = res[1:] - res[:-1]
    return res / k

def calculate_hilbert_spectrum(imfs, t, fs, n=5, plot_hilbert_spec=None, plot_inst_freq=None):
    '''
    Calculate hilbert amplitude spectrum from a given set of intrinsic mode functions.
    '''

    ## Create Hilbert spectrum
    T = t[-1] - t[0]; delta_t = 1 / fs
    fmin = fres = 1 / T; fmax = 1 / (n * delta_t)
    N = int(T / (n * delta_t))
    bin_centres = np.arange(N) * fres + fmin
    bin_edges = np.arange(N + 1) * fres + (fmin - fres / 2)

    hht = np.zeros((len(imfs), N, (len(t) - 2)))

    for j, imf in enumerate(imfs):
        Z = hilbert(imf)
        A = np.abs(Z)
        theta_inst = np.unwrap(np.angle(Z))
        f_inst = np.r_[np.nan,
                       0.5 * (np.angle(-Z[2:] * np.conj(Z[:-2])) + np.pi) / (2 * np.pi) * fs,
                       np.nan]
        t_spec = t[1:-1]; A_spec = A[1:-1]; f_spec = f_inst[1:-1]

        # Plot instantaneous frequency curves
        if plot_inst_freq and plot_inst_freq is not None:
            fig, (ax0, ax1) = plt.subplots(nrows=2)
            ax0.plot(t, imf, label='signal')
            ax0.plot(t, A, label='envelope')
            ax0.set_xlabel("time (s)")
            ax0.set_ylabel("signal (units)")
            ax0.legend()
            ax1.plot(t_spec, f_spec)
            ax1.set_xlabel("time (s)")
            ax1.set_ylabel("frequency (Hz)")
            fig.tight_layout()
            plt.show()

        # Binning of frequency values
        binned_freq = pd.cut(f_spec, bin_edges)
        bin_inds = binned_freq.codes

        # Populate Hilbert spectrum matrix
        for i, bin_ind in enumerate(bin_inds):
            if bin_ind > 0:
                hht[j][bin_ind][i] = A_spec[i]
            else:
                pass

    hht_sum = np.sum(hht, axis=0)
    
    # Downsample HHT
#     hht_sum = downsample_rows(hht_sum, 3)
#     bin_centres = downsample_rows(bin_centres, 3)
    
    # Plot Hilbert spectrum for all IMFs
    if plot_hilbert_spec and plot_hilbert_spec is not None:
        plt.figure()
        plt.pcolormesh(t_spec, bin_centres, hht_sum)
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
        plt.show()
        
    return hht_sum, t_spec, bin_centres

In [3]:
# Import data
fs = 1000
data_path = op.expanduser('~/data/meg/')
x = np.load(op.join(data_path, 'ssvef.npy')); x = x / np.mean(np.abs(x))
# t = np.arange(0, len(x)/fs, 1/fs)
t = np.arange(0, 10001/fs, 1/fs); x = x[:len(t)]

In [4]:
# Perform EMD-HHT
imfs = perform_EMD(x, plot_emd=False)
C = imfs[:-1]
hht, t_hht, f_hht = calculate_hilbert_spectrum(C, t, fs, plot_hilbert_spec=False)

In [5]:
imfs.shape

(12, 10001)

In [6]:
# Marginal spectrum
marginal_spec = np.mean(hht, axis=1)

# Draw marginal spectrum
# plt.figure()
# plt.plot(f_hht, marginal_spec)
# plt.xlim([0, 50])
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Marginal Hilbert Spectrum')
# plt.show()

In [7]:
# Smoothing - Weighted Gaussian Filtering
k_gauss = 15
hht_smooth = cv2.GaussianBlur(hht, (k_gauss, k_gauss), 0)

In [8]:
# Draw HHT
plt.figure()
# plt.pcolormesh(t_hht, f_hht, hht_smooth)
plt.pcolormesh(t_hht, f_hht[:500], hht_smooth[:500, :])
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()

qt.qpa.drawing: Layer-backing can not be explicitly controlled on 10.14 when built against the 10.14 SDK


In [9]:
hht.shape

(2000, 9999)